# Отработка ошибок свзязанных с запуском кода

- Если после установки необходимых библиотек возникает ошибка то скорее всего не установлен вебдрайвер Chrome для вашей ОС. Как установить вебдрайвер можно ознакомиться по этой ссылке. [selenium-python](https://selenium-python.com/install-chromedriver-chrome?ysclid=ly2ufnhjip111135754)
- Если после установки вебдрайвера код не отрабатывает либо возникает ошибка то требуется проверить не открыт ли в памяти браузер Chrome. Если да то требуется его выгрузить (закрыть). Это связано с тем что данный код отрабатывает в фоновом режиме но при этом использует вебдрайвер Chrome который божет конфликтовать с самим запущенным браузером Chrome.
- Если ошибка попрежнему возникает то требуется проверить или переустановить используемые библиотеки.

In [1]:
#!pip3 install --upgrade pip
#!pip3 install selenium
#!pip3 install beautifulsoup4
#!pip3 install pandas
#!pip3 install regex
#!pip3 install requests

In [2]:
import re
import time
import requests
import pandas as pd

from datetime import datetime
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC

In [3]:
class ItikcetUzParser:

    # инициализатор
    def __init__(self, url = 'https://iticket.uz/ru/events'):
        # инициализируем вебдрайвер
        options = Options()
        # гиперпараметр для запуска вебдрайвера селениум без графического интерфейса
        options.add_argument('--headless')
        self.driver = Chrome(options=options)
        # адрес всех событий сайта
        self.url = url
        # параметр максимального времени загрузки страницы сек
        self.upload_wait_time = 10
        # параметр ожидания после прокликивания для загрузки данных для парсинга сек
        self.sleep_time = 1

    # функция для пркликивания элементов страницы и возврата блока парсинга страницы
    def clicker(self):
        self.driver.get(self.url)
        # ожидание загрузки данных
        wait = WebDriverWait(self.driver, self.upload_wait_time)
        counter = 0
        # ждем загрузки конкретных элементов страницы для дальнейшего прокликивания
        wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@class='grid sm:grid-cols-2 lg:grid-cols-3 gap-5 sm:gap-10']//a[@class='event-list-item']")))
        # цикл прокликивания страницы так как после нажатия на кнопку данные прогружаются не полностью, 50 максимальное количество итераций как защита от бесконечного цикла
        while counter < 50:
            try:
                # ожидание, пока элемент станет кликабельным
                span_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'content-around mx-auto btn cursor-pointer')]")))
                # нажатие на искомый элеент
                span_button.click()
                # Небольшая задержка, чтобы избежать слишком частых кликов
                time.sleep(self.sleep_time)
                counter += 1
            except:
                # если элемент больше не кликабелен, выходим из цикла
                break
        # ждем загрузки конкретных элементов страницы после прокликивания
        wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@class='grid sm:grid-cols-2 lg:grid-cols-3 gap-5 sm:gap-10']//a[@class='event-list-item']")))
        # передаем содержимое вебдрайвера после взаимодействия со страницей из селениума в бьютифул суп для парсинга страницы
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        # ищем все ячейки страницы с требуемыми данными, если их нет возвращаем пустой массив
        divs = soup.find('div', attrs={'class':'grid sm:grid-cols-2 lg:grid-cols-3 gap-5 sm:gap-10'}).find_all('a', attrs={'class':'event-list-item'})\
            if soup.find('div', attrs={'class':'grid sm:grid-cols-2 lg:grid-cols-3 gap-5 sm:gap-10'}).find('a', attrs={'class':'event-list-item'}) else []
        return divs

    # функция добавления данных со всей страницы в итоговый фрейм ссылок событий
    def events_page_parser(self, divs):
        df_events_url = pd.DataFrame({'event_name':[], 'page_url':[]})
        if len(divs) > 0:
            for div in divs:
                line = {}
                line['event_name'] = div.get('title') if div.get('title') else None
                event_site = div.get('href') if div.get('href') else None
                line['page_url'] = ('https://iticket.uz' + event_site) if event_site and event_site[0] == '/' else event_site
                df_events_url.loc[len(df_events_url)] = line
        return df_events_url

    # функция проверки элементов на присутствие в блоке, в случае отсутствия добавляет пустое значение в случае присутствия очищает текст от лишних пробелов и табуляции 
    def cleaner(self, text_block):
        text = re.sub(r'\s+', ' ', text_block.text.replace('.', '. ').replace('\n', ' ').replace('\t', ' ')).strip() if text_block else None
        return text

    # функция проверки корректности URL
    def check_url(self, url):
        try:
            # отправляет запрос для проверки статуса url, ответ 200 код существует
            response = requests.head(url)
            if response.status_code == 200:
                return True
            else:
                return False
        except requests.RequestException:
            return False
        
    # функция сбора данных со страницы события
    def page_parser(self, url):
        page_data = {}
        # проверяем URL на факт существования
        if self.check_url(url):
            page_data['url'] = url
        else:
            print('URL не существует')
            return page_data
        page_data['url'] = url
        self.driver.get(url)
        # параметры ожидания загрузки данных
        wait = WebDriverWait(self.driver, self.upload_wait_time)
        try:
            # ожидание загрузки элемента это помогает минимизировать время парсинга не устанавливая фиксированные лимиты ожидания
            wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@class='venue-detail']")))
            wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@id='event-detail']//div[@class='tab-body']")))
            wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@id='imap-parent']//div[@class='accordions']//div[@class='info']")))
        except TimeoutException:
            print("Элемент не был найден в указанное время. Продолжаем выполнение кода.")
        # Небольшая задержка, чтобы избежать недозагруженных данных
        time.sleep(self.sleep_time)
        # передаем содержимое вебдрайвера после загрузки контента из селениума в бьютифул суп для парсинга страницы
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        if soup.find('div', attrs={'id':'tab-0'}):
            page_data['description'] = self.cleaner(soup.find('div', attrs={'id':'tab-0'}).find('div', attrs={'class':'tab-content'}))
        else:
            page_data['description'] = None
        if soup.find('div', attrs={'id':'tab-1'}):
            page_data['age_languages'] = self.cleaner(soup.find('div', attrs={'id':'tab-1'}).find('div', attrs={'class':'tab-content'}))
        else:
            page_data['age_languages'] = None
        # забираем данные мест проведения со страницы
        if soup.find('div', attrs={'class':'venue-detail'}):
            places = soup.find('div', attrs={'class':'venue-detail'}).find_all('div', attrs={'class':'my-10 grid grid-cols-1 lg:grid-cols-12 gap-10'}) \
            if soup.find('div', attrs={'class':'venue-detail'}).find_all('div', attrs={'class':'my-10 grid grid-cols-1 lg:grid-cols-12 gap-10'}) else []
        else:
            places = []
        page_places = []
        for place in places:
            page_place = {}
            page_place['coordinates'] = place.find('div', attrs={'class':'flex-1 map'}).find('iframe', src=True).get('src').split('=')[1].split('&')[0].split(',') \
                if place.find('div', attrs={'class':'flex-1 map'}).find('iframe', src=True) else None
            page_place['place'] = self.cleaner(place.find('div', attrs={'class':'venue-name'}))
            page_place['address'] = self.cleaner(place.find('div', attrs={'class':'venue-address'}))
            # забираем данные блока номеров из ячейки в массив
            phones = place.find_all('div', attrs={'class':'venue-phones'}) if place.find('div', attrs={'class':'venue-phones'}) else []
            # убираем пробелы табуляцию и лишнюю пунктуацию из массива оставляя только цифры и преобразуем в числовой тип данных
            page_place['phone_array'] = [int(re.sub('[^0-9]', '', phone.get_text(strip=True))) for phone in phones]
            page_places.append(page_place)
        page_data['places'] = page_places
        # забираем данные событий со страницы
        if soup.find('div', id='imap-parent'):
            events = soup.find('div', id='imap-parent').find_all('div', attrs={'class':'info'}) if soup.find('div', id='imap-parent').find('div', attrs={'class':'info'}) else []
        else:
            events = []
        page_events = []
        for event in events:
            page_event = {}
            page_event['event_name'] = self.cleaner(event.find('div', attrs={'class':'sa-col-1'}).find('div', attrs={'class':'session-name'}))
            page_event['event_place'] = self.cleaner(event.find('div', attrs={'class':'sa-col-1'}).find('div', attrs={'class':'accordion-label'}))
            page_event['event_cost'] = self.cleaner(event.find('div', attrs={'class':'sa-col-3'}).find('div', attrs={'class':'value'}))
            event_data = event.find('div', attrs={'class':'sa-col-2'}).find('div', attrs={'class':'value'})
            page_event['event_data'] = re.sub(r'\s+', ' ', event_data.text.replace('\n', ' ').replace('\t', ' ')).strip() if event_data else None
            page_events.append(page_event)
        page_data['events'] = page_events
        return page_data

    # функция обхода сылок фрейма событий и парсинг каждой страницы
    def events_site_parser(self):
        df = []
        # вызов парсера страницы событий для формирования фрейма сылок событий
        df_events_url = self.events_page_parser(self.clicker())
        # обход фрейма сылок для прсинга 
        if len(df_events_url) > 0:
            for event_url in df_events_url['page_url']:
                display(event_url)
                df.append(self.page_parser(event_url))
        # закрываем браузер в селениум
        self.driver.quit()
        return pd.DataFrame(df)

In [4]:
class DataConverter:
    
    # инициализатор
    def __init__(self, input_df):
        self.input_df = input_df

    # функция для поиска словаря по ключу и значению
    def find_dict(self, array, key, value):
        for dictionary in array:
            if dictionary.get(key) == value:
                return dictionary
        return None
    
    # функция удаления дубликатов и пустых массивов
    def phone_cleaner(self, phone_array):
        unique_phone_array = list(set(phone_array))
        return unique_phone_array if (len(unique_phone_array) > 0) else None

    # функция конвертации координат в числовой сортированный по возрастанию массив и удаления пустых массивов      
    def coordinates_cleaner(self, coordinates_array):
        try:
            float_coordinates_array = [float(item) for item in coordinates_array]
            sorted_coordinates = sorted(float_coordinates_array)
            return sorted_coordinates
        except ValueError:
            return None

    # функция очистки возраста
    def age_cleaner(self, age_languages):
        age = re.sub('[^0-9]', '', age_languages)
        return int(age) if age else None
    
    # функция языка мероприятия
    def language_cleaner(self, age_languages):
        language = re.sub('[^а-яА-Яa-zA-Z]', '', age_languages)
        return language if (language != '') else None
    
    # функция формирования категории
    def category_cleaner(self, url):
        cleaned_category = url.split('events/', maxsplit=1)[1].split('/', maxsplit=1)[0]
        return cleaned_category
    
    # функция определения регулярности мероприятия критерий 30 дней, стартовой и финальной даты события
    def regularity_event(self, data_text):
        pattern_event = 'regularity_event'
        if data_text:
            split_data_text = re.sub(r'\s+', ' ', (re.sub('[^0-9.:]', ' ', data_text))).strip().split(' ')
            if len(split_data_text) > 1:
                first_data = datetime.strptime((split_data_text[0] + ' ' + split_data_text[1]), '%d.%m.%Y %H:%M')
                if len(split_data_text) == 3:
                    second_data = datetime.strptime((split_data_text[0] + ' ' + split_data_text[2]), '%d.%m.%Y %H:%M')
                elif len(split_data_text) == 4:
                    second_data = datetime.strptime((split_data_text[2] + ' ' + split_data_text[3]), '%d.%m.%Y %H:%M')
                if (second_data - first_data).days < 30:
                    pattern_event = 'not_regularity_event'
        return [pattern_event, first_data, second_data] if pattern_event == 'not_regularity_event' else [pattern_event, None, None]

    # функция для преобразования спарсеных данных в интерпритируемый вид
    def transformation(self):
        output_df = []
        for line in self.input_df.to_dict(orient='records'):    
            if 'events' in line and line['events']:
                for event in line['events']:
                    place_info = self.find_dict(line['places'], 'place', event['event_place'])
                    address = place_info['address'] if place_info else None
                    coordinates = self.coordinates_cleaner(place_info['coordinates']) if place_info else None
                    phone_array = self.phone_cleaner(place_info['phone_array']) if place_info else None
                    new_line = {'url': line['url'], 
                                'category': self.category_cleaner(line['url']), 
                                'description': line['description'], 
                                'age': self.age_cleaner(line['age_languages']),
                                'languages': self.language_cleaner(line['age_languages']),
                                'event_name': event['event_name'], 
                                'place': event['event_place'], 
                                'cost': event['event_cost'], 
                                'data_text': event['event_data'],
                                'pattern_event': self.regularity_event(event['event_data'])[0],
                                'first_data': self.regularity_event(event['event_data'])[1],
                                'second_data': self.regularity_event(event['event_data'])[2],
                                'address': address,
                                'coordinates': coordinates,
                                'phone_array': phone_array
                                }
                    output_df.append(new_line)
            else:
                new_line = {'url': line['url'], 
                            'category': self.category_cleaner(line['url']),
                            'description': line['description'], 
                            'age': self.age_cleaner(line['age_languages']),
                            'languages': self.language_cleaner(line['age_languages']),
                            'event_name': None, 
                            'place': line['places'][0]['place'] if line['places'] else None,
                            'cost': None, 
                            'data_text': None,
                            'pattern_event': self.regularity_event(event['event_data'])[0],
                            'first_data': self.regularity_event(event['event_data'])[1],
                            'second_data': self.regularity_event(event['event_data'])[2],
                            'address': line['places'][0]['address'] if line['places'] else None,
                            'coordinates': self.coordinates_cleaner(line['places'][0]['coordinates']) if line['places'] else None,
                            'phone_array': self.phone_cleaner(line['places'][0]['phone_array']) if line['places'] else None
                            }
                output_df.append(new_line)
        pd_output_df = pd.DataFrame(output_df)
        pd_output_df['age'] = pd_output_df['age'].astype('Int64')        
        return pd_output_df

# Работа с парсером
- Для запуска парсинга сайта [iticket.uz](https://iticket.uz/ru/events) требуется составить запрос с вызовом парсера ItikcetUzParser().events_site_parser() в данном случае парсер соберед нанные сайта с раздела "Все события". При объявлении класса можно задать ссылку из другой категории то парсер соберет данные только по этой категории ItikcetUzParser('https://iticket.uz/ru/events/cultural-events').events_site_parser(). Парсер выдает данные в формате Data Frame Pandas в формате "как собрал."
- Для конвертации даннх написан отдельный класс позволяющий преобразовывать фрейм в легко интерпретируемый Pandas Data Frame. Для вызова требуется составить запрос и при объявлении класса в переменную input_df передать фрейм парсера как в примере DataConverter(parsed_df).transformation().
- Для удобства можно объеденить данные запросы в один запрос который сразу предоставит результирующий фрейм DataConverter(ItikcetUzParser().events_site_parser()).transformation().
- Для сохранения результирующего фрейма в файл CSV требуется раскоментить строку кода cleared_df.to_csv(f"Selenium_Soup_iticket_{datetime.now().strftime('%d%m%Y')}.csv") которая сохранить данные в файл с наименованием "Selenium_Soup_iticket_10072024.csv" а папку с запускаемом кодом. Дата будет подставляться автоматически - текущая дата сохранения результата парсинга.
# Важно
- Парсер не соберает данные событий со страниц событий которые содержат активный элемет id="icalendar" тк без выбора даты не формируется элемент class="accordions" без чего парсинг страницы не имеет смысла. В случае необходимости парсинга такой страницы потребуется дописывть класс парсера с отдельной логикой обработки элемента календаря. На данный момент подобная необходимость отсутствует.

In [5]:
parsed_df = ItikcetUzParser().events_site_parser()

'https://iticket.uz/ru/events/samarkand/sadriddin-ayni-house-museum'

'https://iticket.uz/ru/events/samarkand/house-museum-of-haji-abdulaziz-abdurasulov'

'https://iticket.uz/ru/events/samarkand/mahmudhoja-behbudi-house-museum'

'https://iticket.uz/ru/events/samarkand/rare-devonbegi-madrasah'

'https://iticket.uz/ru/events/samarkand/regional-museum-of-local-lore'

...

'https://iticket.uz/ru/events/concerts/avatar-soundtracks'

'https://iticket.uz/ru/events/concerts/mirzabek-xolmedov-hamda-mirzo-teatr'

'https://iticket.uz/ru/events/concerts/frank-sinatra24'

'https://iticket.uz/ru/events/concerts/christmas-songs-5842'

'https://iticket.uz/ru/events/concerts/yulduz-usmanova2025'

In [6]:
cleared_df = DataConverter(parsed_df).transformation()
display(cleared_df.head(20))
#cleared_df.to_csv(f"Selenium_Soup_iticket_{datetime.now().strftime('%d%m%Y')}.csv")

,url,category,description,age,languages,event_name,place,cost,data_text,pattern_event,first_data,second_data,address,coordinates,phone_array
0,https://iticket.uz/ru/events/samarkand/sadridd...,samarkand,,<NA>,None,Дом-музей Садриддина Айни,Дом-музей Садриддина Айни,2 000 - 30 000 UZS,Ср 28.12.2022 09:00 - Вт 31.12.2024 17:00,regularity_event,NaT,NaT,"Регистан улица, Samarkand, Узбекистан","[39.652637, 66.97255]",None
1,https://iticket.uz/ru/events/samarkand/house-m...,samarkand,Постановлением Кабинета министров Республики У...,<NA>,None,Дом-музей Хаджи Абдулазиза Абдурасулова,Дом-музей Хаджи Абдулазиза Абдурасулова,2 000 - 30 000 UZS,Ср 28.12.2022 09:00 - Вт 31.12.2024 17:00,regularity_event,NaT,NaT,"Регистан улица, Samarkand, Узбекистан","[39.649915, 66.968302]",None
2,https://iticket.uz/ru/events/samarkand/mahmudh...,samarkand,Дом-музей махмудходжи бехбуди был создан 18 ок...,<NA>,None,Дом-музей махмудходжи Бехбуди,Дом-музей махмудходжи Бехбуди,2 000 - 30 000 UZS,Ср 28.12.2022 09:00 - Вт 31.12.2024 17:00,regularity_event,NaT,NaT,,"[39.651583, 66.933765]",None
3,https://iticket.uz/ru/events/samarkand/rare-de...,samarkand,Медресе Надир девонбеги было построено в 1630-...,<NA>,None,Медресе Надир девонбеги,Медресе Надир девонбеги,2 000 - 30 000 UZS,Ср 28.12.2022 09:00 - Вт 31.12.2024 17:00,regularity_event,NaT,NaT,,"[39.6191939, 66.9533669]",None
4,https://iticket.uz/ru/events/samarkand/regiona...,samarkand,Музей был открыт в декабре 1981 года на основе...,<NA>,None,Областной краеведческий музей,Областной краеведческий музей,2 000 - 30 000 UZS,Ср 28.12.2022 09:00 - Вт 31.12.2024 17:00,regularity_event,NaT,NaT,"Абдурахман Жомий улица, Samarkand, Узбекистан","[39.6535351, 66.9538408]",None
5,https://iticket.uz/ru/events/samarkand/state-m...,samarkand,Сведения об основании музея в Самарканде с 187...,<NA>,None,Государственный музей истории культуры Узбекис...,Государственный музей истории культуры Узбекис...,2 000 - 30 000 UZS,Ср 28.12.2022 09:00 - Вт 31.12.2024 17:00,regularity_event,NaT,NaT,"Самарканд, ул. Мирзо Улугбека, 148","[39.666927, 66.927639]",None
6,https://iticket.uz/ru/events/samarkand/history...,samarkand,"В результате исследований, проведенных в течен...",<NA>,None,"""Афросиаб"" Музей истории города Самарканда",Афиябский Музей истории города Самарканда,2 000 - 30 000 UZS,Ср 28.12.2022 09:00 - Вт 31.12.2024 17:00,regularity_event,NaT,NaT,"Тошкент йули, Самарканд, Узбекистан","[39.669249, 66.993541]",None
7,https://iticket.uz/ru/events/samarkand/mirzo-u...,samarkand,Обсерватория Мирзы Улугбека была построена в 1...,<NA>,None,Музей-комплекс Мирзо Улугбека,Музей-комплекс Мирзо Улугбека,2 000 - 30 000 UZS,Ср 28.12.2022 09:00 - Вт 31.12.2024 17:00,regularity_event,NaT,NaT,"Тошкент йули, Самарканд, Узбекистан","[39.6752281, 67.0056371]",None
8,https://iticket.uz/ru/events/samarkand/bibikha...,samarkand,Комплекс бибихоним (мечеть Амира Темура джоме)...,<NA>,None,Комплекс Бибиханим,Комплекс бибиханим,2 000 - 30 000 UZS,Ср 28.12.2022 09:00 - Вт 31.12.2024 17:00,regularity_event,NaT,NaT,город Самарканд И. Улица Каримова,"[39.660387, 66.982435]",None
9,https://iticket.uz/ru/events/samarkand/tomb-of...,samarkand,Мавзолей Амира Темура признан уникальным произ...,<NA>,Русский,Гур Эмир (Язык: Русский),Гур Эмир,2 000 - 30 000 UZS,Вс 1.01.2023 09:00 - Вт 31.12.2024 17:00,regularity_event,NaT,NaT,"1/4, Universitetskiy Boulevard Бўстонсарой кўч...","[39.6485611, 66.9692159]",None
